# Introduction
## Part 2: Quantum Chemistry

In this tutorial, you will learn how to run basic quantum chemistry
calculations.
Quantum chemistry uses quantum mechanics to predict chemical properties.

### The Born-Oppenheimer Approximation

A fundamental simplifying assumtion in most (not all) quantum chemistry
calculations is the
[Born-Oppenheimer approximation](https://en.wikipedia.org/wiki/Born%E2%80%93Oppenheimer_approximation),
which assumes that the electrons in a molecule move so quickly that they
rearrange almost instantaneously with any nuclear motion.
This is sometimes called the "flies on an ox model."
This allows the molecular
[Schrödinger equation](https://en.wikipedia.org/wiki/Schr%C3%B6dinger_equation)
to be separated into two simpler equations:
$$
\begin{align*}
    \hat{H}_\text{e}(\mathbf{r}_\text{n})
    \Psi_\text{e}(\mathbf{r}_\text{e}; \mathbf{r}_\text{n})
    =&
    E_\text{e}(\mathbf{r}_\text{n})
    \Psi_\text{e}(\mathbf{r}_\text{e}; \mathbf{r}_\text{n})
    \\
    (\hat{T}_\text{n} + E_\text{e}(\mathbf{r}_\text{n}))
    \Psi_\text{n}(\mathbf{r}_\text{n})
    =& E_\text{n}
    \Psi_\text{n}(\mathbf{r}_\text{n})
\end{align*}
$$

If you have not taken a physical chemistry or quantum mechanics course, these
equations may not make much sense to you.
For our purposes, you only need a qualitative understanding of the
Born-Oppenheimer approximation:
Qualitatively, these equations mean that the total energy of the electrons or
*electronic energy*,
$E_\text{e}(\mathbf{r}_\text{n})$,
defines the potential energy of the nuclei.
It is this *potential energy surface* (PES) that governs nuclear motion and is
fundamental to all of quantum chemistry.
The general workflow in quantum chemistry is therefore a two-step process:
1. Calculate relevant points on the PES of the molecule(s) of interest. These are called *electronic structure calculations*.
2. Use these energies to determine properties of a molecule as a whole.

To see how this connects with things you may have seen before, consider a reaction energy diagram:
![reaction energy diagram](../../.github/chemistry-energy-diagram.svg)

This diagram represents a slice of the PES for a reaction.
If we can determine the sequence of molecular structures or "geometries"
connecting the reactants to the products for this reaction, we can calculate the
above energy diagram by running an electronic structure calculation for each of
these points.
This information, in turn can be used to calculate properties of the reaction, such as the reaction rate.

####

### `protomol`

[`protomol`](../../src/protomol/) is a helper module that goes along with
these programming projects and provides some convenience functions for
interacting with various cheminformatics libraries. You will be modifying and
adding to `protomol` as you go along, so it is worth getting familiar with the
basic structure.

`protomol` is organized as a series of submodules, including:
 - [geom](../../src/protomol/geom.py): Functions for working with molecular structures or "geometries"
 - [smiles](../../src/protomol/smiles.py): Functions for working with [SMILES strings](https://en.wikipedia.org/wiki/Simplified_Molecular_Input_Line_Entry_System#Examples)
 - [rd.mol](../../src/protomol/rd/mol.py): Functions for working with the `Mol` objects of the cheminformatics library [RDKit](https://www.rdkit.org/docs/GettingStartedInPython.html)

Each of these submodules consists of functions acting on the given data type,
along with a series of "constructor" functions for creating the data type from
another.
The constructor functions are all named `from_<other data type name>`.

The following example shows how you might use these submodules.
It converts a SMILES string to a `Geometry` data structure and extracts various
information for it.

In [1]:
from protomol import smiles, geom

smi = "C[CH2]"  # SMILES formula for propyl radical

# Generate a geometry for the SMILES formula
geo = smiles.geometry(smi)

# Get information from the geometry data structure
natms = geom.count(geo)
syms = geom.symbols(geo)
xyzs = geom.coordinates(geo)
char = geom.charge(geo)
spin = geom.spin(geo)

print("Atom count:", natms)
print("Symbols:", syms)
print("Coordinates (in Angstrom):", xyzs)
print("Charge:", char)
print("Spin:", spin)

Atom count: 7
Symbols: ['C', 'C', 'H', 'H', 'H', 'H', 'H']
Coordinates (in Angstrom): [[-1.15183444e+00  1.58276749e-03 -8.91999558e-03]
 [ 1.63023378e+00 -1.41875685e-01 -5.63693033e-01]
 [-2.10893957e+00  9.31392854e-01 -1.65090989e+00]
 [-1.48788168e+00  1.22589432e+00  1.67560268e+00]
 [-1.99239882e+00 -1.88989478e+00  3.33461032e-01]
 [ 2.53284069e+00  1.66509626e+00 -5.15261351e-02]
 [ 2.57798004e+00 -1.79219573e+00  2.65985341e-01]]
Charge: 0
Spin: 1


In a Jupyter notebook, you can also display the molecular geometry as follows.

In [2]:
geom.display(geo)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Initially, we will be primarily focused on the `Geometry` data structure of the `geom` submodule, which has the following attributes:
 - `symbols`: The list of atomic symbols
 - `coordinates`: A $n\times3$ numpy array of coordinates in (units: [Bohr radii](https://en.wikipedia.org/wiki/Bohr_radius))
 - `charge`: The total charge of the molecule
 - `spin`: The total spin of the electrons in the molecule ($2S$ where $S$ is the total spin quantum number)

You can print the data structure to see the values of these attributes.

In [3]:
print(geo)

symbols=['C', 'C', 'H', 'H', 'H', 'H', 'H'] coordinates=array([[-1.15183444e+00,  1.58276749e-03, -8.91999558e-03],
       [ 1.63023378e+00, -1.41875685e-01, -5.63693033e-01],
       [-2.10893957e+00,  9.31392854e-01, -1.65090989e+00],
       [-1.48788168e+00,  1.22589432e+00,  1.67560268e+00],
       [-1.99239882e+00, -1.88989478e+00,  3.33461032e-01],
       [ 2.53284069e+00,  1.66509626e+00, -5.15261351e-02],
       [ 2.57798004e+00, -1.79219573e+00,  2.65985341e-01]]) charge=0 spin=1


While these values can be accessed directly as `geo.symbols`, `geo.coordinates`,
etc., it is recommended to use the functions above to access these values
instead.
These functions can be used to process the values for convenience, such as converting the units of the coordinates.

In [4]:
xyzs_bohr = geom.coordinates(geo)  # equivalent to geo.coordinates
xyzs_angs = geom.coordinates(geo, unit="angstrom")

print("Coordinates in Bohr:")
print(xyzs_bohr)  

print("Coordinates in Angstroms:")
print(xyzs_angs)

Coordinates in Bohr:
[[-1.15183444e+00  1.58276749e-03 -8.91999558e-03]
 [ 1.63023378e+00 -1.41875685e-01 -5.63693033e-01]
 [-2.10893957e+00  9.31392854e-01 -1.65090989e+00]
 [-1.48788168e+00  1.22589432e+00  1.67560268e+00]
 [-1.99239882e+00 -1.88989478e+00  3.33461032e-01]
 [ 2.53284069e+00  1.66509626e+00 -5.15261351e-02]
 [ 2.57798004e+00 -1.79219573e+00  2.65985341e-01]]
Coordinates in Angstroms:
[[-6.09524539e-01  8.37564487e-04 -4.72025838e-03]
 [ 8.62682566e-01 -7.50773794e-02 -2.98293507e-01]
 [-1.11600276e+00  4.92871873e-01 -8.73623891e-01]
 [-7.87353080e-01  6.48715336e-01  8.86690753e-01]
 [-1.05433205e+00 -1.00008925e+00  1.76459979e-01]
 [ 1.34032157e+00  8.81130994e-01 -2.72664564e-02]
 [ 1.36420829e+00 -9.48389139e-01  1.40753381e-01]]


### Next step: complete [exercise.ipynb](exercise.ipynb)